In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
Scipy not supported!


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
['addiction', 'affirmation_distraction_addiction', 'anxiety', 'challenge_thought_anxiety', 'depression', 'explain_distraction_addiction', 'explainreccdist_depression', 'first_substance_addiction', 'go_plan_for_things', 'goodbye', 'greeting', 'how_wreck', 'ideas_wrecking', 'need_help', 'need_help_adiction', 'need_ideas_addiction', 'no_distraction_addiction', 'no_distraction_addiction_after', 'plan_for_thing', 'ques_to_q1_anxiety', 'question1_anxiety', 'question1_depression', 'question_to_question1_depression', 'question_to_question2_anxiety', 'question_to_question2_depression', 'question_to_question3_depression', 'questtion_to_question4_depression', 'stating_problem', 'thanks', 'the_end', 'the_end_bad', 'things_addiction', 'what_to_do_addiction2', 'work_on_it', 'wo

In [6]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\samit\MACHINE LEARNING\chatbot BE project\New\model.tflearn


In [15]:
yeah_dict = {'problem_got': ('categorize_problem', 'How would you categorize this issue 1.Depression, 2.Anxiety, 3.Addiction, 4.Angry, 5.Grief'), 'depressed':('depressed_1', 'So do you want my help with it or you just wanted to share?'), 'write_thought': ('questoques1recognized','Good job recognizing. Labels can weigh us down and squash our growth mindset') , 'catastrophe': ('makingUnderstandques2', 'Its like looking at your problems through a binoculars. But if you lower the binoculars you might see your problems from a more realistic point of view'), 'write_thought_anxiety': ('questoques1confirmed_anxiety','Thats common....but does not mean its good way to deal with pressure'), 'future_assuming':('future_assuming_explanation', 'You assume, if you attempt something it will turn out poorly'), 'distortion': ('gotchaques3', 'yeah with this distortion it can feel like destiny is weighing us down. This is called Overgeneralisation.'), 'emotionalreasoning': ('rewritethoughts','Nice spot!'), 'future_assuming': ('affirmation_anxiety','You need to understand you can not foresee the future'), 'distraction_addiction':('done_solution', 'sure, distraction it is (take to any site with fun activities or give them riddles)')}
no_dict = {'problem_told_depression': ('catastrophe','Does Your Thought make a catastrophe out of a situation?'), 'catastrophe': ('distortion', 'Does it assume a continuing patterns of negativity or a never ending cycle of defeat?'),'distortion': ('emotionalreasoning', 'Does your thought suggest any emotional reasoning?'), 'write_thought_anxiety':('future_assuming', 'Is your thought future assuming?'), 'future_assuming': ('mind_reading', 'Is your thought mind reading?'), 'emotionalreasoning': ('rewritethoughts', 'Ok now the most crucial peice... please rewrite your negative thoughts so that its more realistic. what would be a kinder more balanced way to talk to yourself here?'), 'ideas_wrecking':('which_category', 'Sure ive got tips for handling relationships, places, things which would be helpful? We can do more than one')}
okay_dict = {'depressed':('depressed_1', 'So do you want my help with it or you just wanted to share?'),'problem_got': ('categorize_problem','How would you categorize this issue 1.Angry, 2.Anxiety, 3.Depression'),'questoques1explained':('write_thought', 'So let me ask you again, does the thought attach any labels to you?'), 'questoques1recognized':('catastrophe', 'Does Your Thought make a catastrophe out of a situation?'), 'ques2explained': ('catastrophe', 'So let me ask you again, Does Your Thought make a catastrophe out of a situation?'),'makingUnderstandques2': ('distortion', 'Does it assume a continuing patterns of negativity or a never ending cycle of defeat?'),'ques3explained':('givingAName', 'This is often a recurring distortion for perfectionists'), 'everythingExplained':('distortion', 'Let me ask another way: are you assuming that the way something has turned out now, is also the way things will turn out in the future'), 'gotchaques3':('emotionalreasoning', 'Does your thought suggest any emotional reasoning?'), 'ques4explained':('emotionalreasoning', 'sO WHAT DO YOU THINK IS THERE ANY emotional reasoning'), 'questoques1explained_anxiety':('write_thought_anxiety', 'So let me ask you again, does the thought serve a purpose?'), 'questoques1confirmed_anxiety':('future_assuming', 'Is your thought future assuming?'), 'future_assuming_explanation':('future_assuming', 'So let me ask you again, is your thought future assuming?'), 'affirmation_anxiety':('mind_reading', 'Is your thought mind reading?'), 'addiction':('what_to_do_adiction', 'OK, did you want my help with this right now, or did you just want me to know about it?'), 'help_addiction':('distraction_addiction', 'well first up we could try a distraction'), 'explain_distraction':('distraction_addiction', 'it wont solve the problem but it can help dial down the intensity of a craving'), 'distraction_addiction':('done_solution', 'sure, distraction it is (take to any site with fun activities or give them riddles)'), 'no_distraction':('no_distraction2', 'first we ll do a guided imagery exercise together, imaging your substance as an island. Then we ll identify everything that allows you to keep going back. Those are the ships we ll be wrecking.'), 'no_distraction2':('no_distraction3','then Ill ask you to do the hard ad important work of committing to wrecking those ships'), 'wrecking_explained':('first_substance', 'okay buckle in! First whats the substance you want to change your relationship with?'), 'got_first_substance': ('look_island', 'think about what the island looks like'), 'look_island': ('who_island', 'imagine whos on the island with you. maybe you are alone.'), 'who_island': ('ways_island', 'Now think of all the ways you get to your island. Like where you use it, the people you use with, all the equipments or materials you associate with it, and everything that triggers cravings'), 'ways_island':('write_ships','write it all here'), 'wrote_ships':('destroy_ships', 'I want you to envision your ships colliding. Hear the crash and the creak of wood. and watch them spin down in a whirlpool until they are sucked to the bottom of the sea out of your grasp'), 'destroy_ships':('next_job','Great job with the imagery we ve come a long way and we still have some work ahead of us'), 'next_job':('ideas_wrecking', 'So do you have any ideas about a strategy for wrecking your ships'), 'solve_things':('sharing_plan_for_things', 'ok share your plan with me here'), 'last_challenge_things':('the_end_things', 'i know this was a long exercise so thanks for sticking it out nd making a plan with me. How are you feeling after meeting this strategy?'), 'no_distraction3':('first_substance', 
'okay buckle in! First whats the substance you n=want to change your relationship with?'), '':('','')}

In [17]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(userID='123', show_details=True):
    while True:
        resp = ''
        sentence = input()
        if sentence == 'bye':
            break
        results = classify(sentence)
        r1 = results[0]
        try:
            past_context = context[userID]
        except:
            pass
        try:            # find a tag matching the first result
            if sentence in ['yes','yeah']:
                #print('in yeah loop')
                tple = yeah_dict[context[userID]]
                #print(tple)
                resp = tple[1]
                context[userID] = tple[0]
                    #if i['tag'] == tag:
                        #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #context[userID] = i['context_set']
                        #print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                print('first yeah  ',resp)
                continue
        except:
            #print('context_set ',context[userID],' past_context ',past_context)
            print('sorry I didnt get you. Will you please be a little clear')
            continue
        try:
            if sentence in ['no','nope', 'not at all']:
                #print('in no loop')
                tple = no_dict[context[userID]]
                #print(tple)
                resp = tple[1]
                context[userID] = tple[0]
        #if i['tag'] == tag:
            #resp = random.choice(i['responses'])
            #if 'context_set' in i:
                #context[userID] = i['context_set']
            #print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                print('first  ',resp)
                continue
        except:
            #print('context_set ',context[userID],' past_context ',past_context)
            print('sorry I didnt get you. Will you please be a little clear')
            continue

        try:
            if sentence in ['okay','cool','true','sure', 'gotcha', 'it is', 'got it', 'alright', 'it does']:
                #print('in okay loop')
                
                print('context_filter ', context[userID])
                tple = okay_dict[context[userID]]
                if tple[0] == 'wrecking_explained' or tple[0] == 'no_distraction3':
                    
                    substance = input('okay buckle in! First whats the substance you want to change your relationship with? ')
                    print('I want you to think of using it as an island')
                    context[userID] = 'got_first_substance'
                    continue
                elif tple[0] == 'ways_island':
                    ships = input('write it all here ')
                    print('these are the ships you are wrecking')
                    context[userID] = 'wrote_ships'
                    continue
                else:                    
                #print(tple)
                    resp = tple[1]
                    context[userID] = tple[0]
                    print('new context ', context[userID])
        #if i['tag'] == tag:
            #resp = random.choice(i['responses'])
            #if 'context_set' in i:
                #context[userID] = i['context_set']
            #print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                print('first  ',resp)
                continue
        except:
            #print('context_set ',context[userID],' past_context ',past_context)
            print('sorry I didnt get you. Will you please be a little clear')
            continue
    
    
    
    #Problem input
        if r1[0] == 'need_help':
            context[userID] = 'problem_got'
            problem = input("okay tell me your problem ")
            print("Ouch that sucks")
            continue
        
    
    
        
    # if we have a classification then find the matching intent tag
        if results:
            print(results)
        # loop as long as there are matches to process
            try:
                for i in results:
                    print(i)
                
                    for j in intents['intents']:
                        if i[0]==j['tag']:
                            if not 'context_filter' in j or \
                                (userID in context and 'context_filter' in j and j['context_filter'] == context[userID]):
                                print('in second if')
                                if show_details: print ('in first if tag:', j['tag'])
                                resp = random.choice(j['responses'])
                                if 'context_set' in j: 
                                    context[userID] = j['context_set']
                                    print('new context  ', context[userID])
                if resp == '':
                    print('sorry I didnt get you. Will you please be a little clear')
                    continue
                else: 
                    print('fourth ',resp)
                    continue
                                
            except:
                    #try:
                        #print('context_set ',context[userID],' past_context ',past_context)
                    #except:
                        #pass
                    print('sorry I didnt get you. Will you please be a little clear')
                    continue

        

else:
                    print('in else loop')
                    if 'context_filter' in j and j['context_filter'] == context[userID]:
                        resp = random.choice(j['responses'])
                        print('tag: ', j['tag'])
                        if 'context_set' in j: 
                            context[userID] = j['context_set']
                            print('new context  ', context[userID])
                        return(resp)

while results:
            for i in intents['intents']:
               
                    if i['tag'] == results[0][0]:
                        print('past context ',context, ' tag:', i['tag'])

                        if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                            if show_details: print ('in first if tag:', i['tag'])
                            resp = random.choice(i['responses'])

                        if 'context_filter' in i:
                            if show_details: print('context_filter in second if : ', i['context_filter'])
                            resp = random.choice(i['responses'])

                            # set context for this intent if necessary
                        if 'context_set' in i:
                            if show_details: print ('context in third if : ', i['context_set'])
                            context[userID] = i['context_set']
                            print('new context ', context)

                        if 'context_filter' in i:
                            if show_details: print('context_filter in fourth if : ', i['context_filter'])


                        return print('second  ' ,resp)


In [18]:
response()

hi
[('greeting', 0.9715257)]
('greeting', 0.9715257)
in second if
in first if tag: greeting
fourth  Hello, How can I help you?
help me feel better
okay tell me your problem ond sadj as
Ouch that sucks
yes
first yeah   How would you categorize this issue 1.Depression, 2.Anxiety, 3.Addiction, 4.Angry, 5.Grief
addiction
[('addiction', 0.9913141)]
('addiction', 0.9913141)
in second if
in first if tag: addiction
new context   addiction
fourth  I know this can be really difficult process and im here to help you through the next few moments and long term as well
okay
context_filter  addiction
new context  what_to_do_adiction
first   OK, did you want my help with this right now, or did you just want me to know about it?
help
[('need_help_adiction', 0.3689158)]
('need_help_adiction', 0.3689158)
in second if
in first if tag: need_help_adiction
new context   help_addiction
fourth  okay there are few ways we can go about it
what
[('what_to_do_addiction2', 0.9299053)]
('what_to_do_addiction2', 0.92

KeyboardInterrupt: Interrupted by user

In [ ]:
response('help me feel better')

In [ ]:
response('im having problem with')

In [ ]:
response('it does')

In [ ]:
response('depression')

In [ ]:
response('catastrophe')

In [ ]:
response('catastrophe?')

In [ ]:
response('yes')

In [ ]:
response('want your help')

In [ ]:
response('labels?')

In [ ]:
response('challenge thought')

In [ ]:
response('okay')

In [ ]:
response('got it')

In [ ]:
response('catastrophe?')

In [ ]:
response('okay')

In [ ]:
response('yes')

In [ ]:
response('hi')

In [ ]:
response('anxiety')

In [ ]:
response('work on it')

In [ ]:
response('challenge my thought')

In [ ]:
response('i have this problem')

In [ ]:
response('what is that?')

In [ ]:
response('got it')

In [ ]:
response('yes')

In [ ]:
response('okay')

In [ ]:
response('no')

In [ ]:
response('no')